In [1]:
seed=1
from deeppavlov import build_model
from deeppavlov import configs
import pandas as pd
import os
import numpy as np
import joblib
import sklearn.cluster as cluster
import umap
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
pd.set_option('mode.chained_assignment', None)
from elmo_clustering import ELMoClustering

## Данные

In [2]:
data_path = 'data'

In [3]:
if not os.path.exists('outs'):
    os.mkdir('outs')

In [4]:
test = pd.read_csv(os.path.join(data_path, 'main/wiki-wiki/test-solution.csv'), sep='\t')
test.head()

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,440,банка,1,NaN,"7-12, 36-41",здание банка китая ( макао ) здание банка кита...
1,441,банка,2,NaN,"13-18, 201-206",трехлитровая банка во времена ссср такие банки...
2,442,банка,1,NaN,"21-26, 58-63, 101-106, 263-268, 538-543",здание национального банка украины в здании на...
3,443,банка,1,NaN,"134-139, 262-267, 345-350",зюдфиртель города эссен ( федеральная земля се...
4,444,банка,1,NaN,"43-48, 300-305, 381-386, 521-526",ведения боевых действий полевые учреждения бан...


## Модель

Векторные представления слов с учетом контекста можно получить из языковой модели ELMo. Векторное представление контекста получалось усреднением векторов отдельных слов. Размерность векторов ELMo большая - 2560. Использование алгоритма сокращения размерности UMAP позволило улучшить результат.

In [5]:
model = build_model(configs.embedder.elmo_ru_wiki, download=True)

2020-09-01 17:50:42.291 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /home/laks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/laks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/laks/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/laks/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!





INFO:tensorflow:Saver not created because there are no variables in the graph to restore



In [6]:
if not os.path.exists('vecs'):
    os.mkdir('vecs')

In [7]:
vecs = model(test.context.to_list()) # сохраним вектора, чтобы не тратить время на их построение 
joblib.dump(vecs, 'vecs/test-elmo-vecs.joblib') # при перезапуске и для воспроизводимости результатов

['vecs/test-elmo-vecs.joblib']

## Результаты на wiki-wiki

In [8]:
kmeans_params={'n_clusters':2, 'n_init':20, 'random_state':seed}
#количество кластеров - из описания данных http://www.dialog-21.ru/media/4539/panchenkoaplusetal.pdf

umap_params = {'n_neighbors':20,
              'min_dist':0.03,
              'metric':'cosine',
              'random_state':seed}
elmo_clustering = ELMoClustering(data_path=os.path.join(data_path, 'main/wiki-wiki/test-solution.csv'),
                                          vecs_path='vecs/test-elmo-vecs.joblib',
                                          seed=seed,
                                          umap_params=umap_params,
                                          kmeans_params=kmeans_params,
                                          out_path='outs/elmo-test-wiki-wiki.csv',
                                          ari_venv_path='../../deeppavlovenv')
elmo_clustering.fit_predict()

# запускает в subprocess скрипт из соревнования.
ari=elmo_clustering.evaluate()
print(ari)

word	ari	count
банка	0.704805	112
белка	0.816314	135
бит	1.000000	84
горе	0.868894	60
граф	0.972907	148
душ	0.000000	79
	0.759327	618

